# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import psycopg2.extras
import pandas as pd
from sql_queries import *
import  create_tables

In [2]:
create_tables.main()

In [3]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()
# conn.set_session(autocommit=True)

In [4]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [5]:
song_data_path = './data/song_data'
song_files = get_files(song_data_path)

In [6]:
# Read all song files into one dataframe
df = pd.concat((pd.read_json(file, lines=True) for file in song_files))
df

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,AR7SMBG1187B9B9066,NaN,NaN,,Los Manolos,SOBCOSW12A8C13D398,Rumba De Barcelona,218.38322,0
0,1,ARBGXIG122988F409D,37.77916,-122.42005,California - SF,Steel Rain,SOOJPRH12A8C141995,Loaded Like A Gun,173.19138,0
0,1,ARNNKDK1187B98BBD5,45.80726,15.96760,Zagreb Croatia,Jinx,SOFNOQK12AB01840FC,Kutt Free (DJ Volume Remix),407.37914,0
0,1,ARP6N5A1187B99D1A3,NaN,NaN,"Hamtramck, MI",Mitch Ryder,SOXILUQ12A58A7C72A,Jenny Take a Ride,207.43791,2004
0,1,AR36F9J1187FB406F1,56.27609,9.51695,Denmark,Bombay Rockers,SOBKWDJ12A8C13B2F3,Wild Rose (Back 2 Basics Mix),230.71302,0
...,...,...,...,...,...,...,...,...,...,...
0,1,AR47JEX1187B995D81,37.83721,-94.35868,"Nevada, MO",SUE THOMPSON,SOBLGCN12AB0183212,James (Hold The Ladder Steady),124.86485,1985
0,1,ARNTLGG11E2835DDB9,NaN,NaN,,Clp,SOZQDIU12A58A7BCF6,Superconfidential,338.31138,0
0,1,AR1Y2PT1187FB5B9CE,27.94017,-82.32547,Brandon,John Wesley,SOLLHMX12AB01846DC,The Emperor Falls,484.62322,0
0,1,ARH4Z031187B9A71F2,40.73197,-74.17418,"Newark, NJ",Faye Adams,SOVYKGO12AB0187199,Crazy Mixed Up World,156.39465,1961


In [7]:
# Drop rows where the song_id is identical - Not really necessary
df = df.drop_duplicates(subset='song_id', keep='last')

## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [8]:
columns = ['song_id', 'title', 'artist_id', 'year', 'duration']
song_data = list(df[columns].values[0])
song_data

['SOBCOSW12A8C13D398',
 'Rumba De Barcelona',
 'AR7SMBG1187B9B9066',
 0,
 218.38322]

In [9]:
title_list = list(df['title'])

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [10]:
df_songs_data = df[columns]

In [11]:
df_songs_data

,song_id,title,artist_id,year,duration
0,SOBCOSW12A8C13D398,Rumba De Barcelona,AR7SMBG1187B9B9066,0,218.38322
0,SOOJPRH12A8C141995,Loaded Like A Gun,ARBGXIG122988F409D,0,173.19138
0,SOFNOQK12AB01840FC,Kutt Free (DJ Volume Remix),ARNNKDK1187B98BBD5,0,407.37914
0,SOXILUQ12A58A7C72A,Jenny Take a Ride,ARP6N5A1187B99D1A3,2004,207.43791
0,SOBKWDJ12A8C13B2F3,Wild Rose (Back 2 Basics Mix),AR36F9J1187FB406F1,0,230.71302
...,...,...,...,...,...
0,SOBLGCN12AB0183212,James (Hold The Ladder Steady),AR47JEX1187B995D81,1985,124.86485
0,SOZQDIU12A58A7BCF6,Superconfidential,ARNTLGG11E2835DDB9,0,338.31138
0,SOLLHMX12AB01846DC,The Emperor Falls,AR1Y2PT1187FB5B9CE,0,484.62322
0,SOVYKGO12AB0187199,Crazy Mixed Up World,ARH4Z031187B9A71F2,1961,156.39465


In [12]:
# %time
# for _, row in df_songs_data.iterrows():
#     # print(tuple(row.values))
#     # continue
#     try:
#         cur.execute(song_table_insert, tuple(row.values))
#         conn.commit()
#     except psycopg2.Error as e:
#         print("Error while batch-inserting the song data:")
#         print(e)
# CPU times: user 3 µs, sys: 1 µs, total: 4 µs
# Wall time: 6.2 µs

In [13]:
def insert_df_into_table(cur, sql_query, df):
    """Inserting values in a dataframe into the database.

    Inserts values from a dataframe into a table. 
    The dataframe can only contain the columns to be inserted.

    Args:
        cur (psycopg2.extensions.cursor): Cursor to the postgres database.
        sql_query (str): String with the sql query.
        df (pandas.Dataframe): Dataframe with the data to insert into the table.
    
    Example:
        Inserting song data into a songs table with the columns
        (song_id, title, artist_id, year, duration)
        sql_query = "INSERT INTO songs (song_id, title, artist_id, year, duration)\
                     VALUES %s\
                     ON CONFLICT (song_id)\
                     DO UPDATE SET title = EXCLUDED.title,\
                                   artist_id = EXCLUDED.artist_id,\
                                   year = EXCLUDED.year,\
                                   duration = EXCLUDED.duration;\
        df = pandas.Dataframe with the same columns and same data types.
    """
    data = [tuple(ii) for ii in df.to_numpy()]

    try:
        psycopg2.extras.execute_values(cur, sql_query, data)
        conn.commit()
    except psycopg2.Error as e:
        print("Error while inserting pandas.Dataframe into the table:")
        print(e)

In [14]:
%time
# song_data = [tuple(ii) for ii in df[columns].to_numpy()]

# try:
#     psycopg2.extras.execute_values(cur, song_table_batch_insert, song_data)
#     conn.commit()
# except psycopg2.Error as e:
#     print("Error while batch-inserting the song data:")
#     print(e)
insert_df_into_table(cur, song_table_batch_insert, df[columns])
conn.commit()
# CPU times: user 2 µs, sys: 0 ns, total: 2 µs
# Wall time: 5.48 µs

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 7.15 µs


Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [15]:
df.columns

Index(['num_songs', 'artist_id', 'artist_latitude', 'artist_longitude',
       'artist_location', 'artist_name', 'song_id', 'title', 'duration',
       'year'],
      dtype='object')

In [16]:
artist_columns = ['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']

df_artist_data = df[artist_columns]
df_artist = df_artist_data.drop_duplicates(subset='artist_id', keep='last')

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [17]:
insert_df_into_table(cur, artist_table_batch_insert, df_artist_data)
conn.commit()

Error while inserting pandas.Dataframe into the table:
ON CONFLICT DO UPDATE command cannot affect row a second time
HINT:  Ensure that no rows proposed for insertion within the same command have duplicate constrained values.



Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [19]:
log_files_path = 'data/log_data'
log_files = get_files(log_files_path)

In [20]:
df = pd.concat((pd.read_json(file, lines=True) for file in log_files), axis=0)

## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [21]:
df = df[df['page'] == 'NextSong']

In [22]:
t = pd.to_datetime(df['ts'], unit='ms', origin='unix')
# t.head()

In [23]:
time_data = list(zip(t, t.dt.hour, t.dt.day, t.dt.isocalendar().week, t.dt.month, t.dt.year, t.dt.weekday))
column_labels = ['timestamp', 'hour', 'day', 'week', 'month', 'year', 'weekday']

In [24]:
df_time = pd.DataFrame(time_data, columns=column_labels)
df_time = df_time.drop_duplicates()
df_time.head()

,timestamp,hour,day,week,month,year,weekday
0,2018-11-21 00:57:58.796,0,21,47,11,2018,2
1,2018-11-21 00:58:41.796,0,21,47,11,2018,2
2,2018-11-21 01:02:05.796,1,21,47,11,2018,2
3,2018-11-21 01:03:13.796,1,21,47,11,2018,2
4,2018-11-21 01:05:29.796,1,21,47,11,2018,2


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [25]:
insert_df_into_table(cur, time_table_batch_insert, df_time)
conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [26]:
user_columns = ['userId', 'firstName', 'lastName', 'gender', 'level']
df_user = df[user_columns]

In [27]:
# check data quality

df = df.dropna(axis=0)

df = df[df['userId'] != '']

df_user['userId'] = df_user['userId'].astype(int)
df_user = df_user.drop_duplicates(subset='userId', keep='last')


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [28]:
insert_df_into_table(cur, user_table_batch_insert, df_user)
conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [29]:
df.columns

Index(['artist', 'auth', 'firstName', 'gender', 'itemInSession', 'lastName',
       'length', 'level', 'location', 'method', 'page', 'registration',
       'sessionId', 'song', 'status', 'ts', 'userAgent', 'userId'],
      dtype='object')

In [34]:
df['songplay_id'] = df['sessionId'].astype(str) + '-' + df['ts'].astype(str)
df['start_time'] = pd.to_datetime(df.ts, unit='ms', origin='unix')
songplay_columns = ['songplay_id', 'start_time', 'userId', 'level', 'song', 
                    'artist', 'sessionId', 'location', 'userAgent', 'length']
df_songplay = df[songplay_columns].reset_index(drop=True)
df_songplay
# insert into temporary table
cur.execute(temp_log_data_create)
conn.commit()
insert_df_into_table(cur, sql_insert_temp_table, df_songplay)
cur.execute(join_log_data_songs_artists)
conn.commit()

In [35]:
# %time
# insert_df_into_table(cur, sql_insert_temp_table, df_in)
# conn.commit()

In [36]:
cur.execute("""
select 
    tld.songplay_id, 
    tld.start_time, 
    tld.user_id, 
    tld.level, 
    s.song_id, 
    a.artist_id, 
    tld.session_id, 
    tld.location, 
    tld.user_agent
from temp_log_data tld
left join songs s on
    tld.song = s.title
left join artists a on
    tld.artist = a.name
 """)
conn.commit()
for ii in range(0,5):
    row = cur.fetchone()
    if row == None:
        break
    print(row)
conn.commit()

('774-1542761878796', datetime.datetime(2018, 11, 21, 0, 57, 58, 796000), 80, 'paid', None, None, '774', 'Portland-South Portland, ME', '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"')
('671-1542761921796', datetime.datetime(2018, 11, 21, 0, 58, 41, 796000), 97, 'paid', None, None, '671', 'Lansing-East Lansing, MI', '"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/37.0.2062.94 Safari/537.36"')
('671-1542762125796', datetime.datetime(2018, 11, 21, 1, 2, 5, 796000), 97, 'paid', None, None, '671', 'Lansing-East Lansing, MI', '"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/37.0.2062.94 Safari/537.36"')
('774-1542762193796', datetime.datetime(2018, 11, 21, 1, 3, 13, 796000), 80, 'paid', None, None, '774', 'Portland-South Portland, ME', '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [37]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.

In [38]:
print("finished")

finished
